In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [9]:
ipl2020=pd.read_csv('C:/Users/Admin/Desktop/BYOP Project/IPL Ball-by-Ball 2008-2020.csv')
ipl2021=pd.read_csv('C:/Users/Admin/Desktop/BYOP Project/IPL Ball-by-Ball Dataset 2021.csv')


In [10]:
matchdata = pd.concat(
    map(pd.read_csv, ['C:/Users/Admin/Desktop/BYOP Project/IPL Ball-by-Ball 2008-2020.csv', 'C:/Users/Admin/Desktop/BYOP Project/IPL Ball-by-Ball Dataset 2021.csv']))

In [11]:
matchdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198569 entries, 0 to 5100
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                198569 non-null  int64 
 1   inning            198569 non-null  int64 
 2   over              198569 non-null  int64 
 3   ball              198569 non-null  int64 
 4   batsman           198569 non-null  object
 5   non_striker       198569 non-null  object
 6   bowler            198569 non-null  object
 7   batsman_runs      198569 non-null  int64 
 8   extra_runs        198569 non-null  int64 
 9   total_runs        198569 non-null  int64 
 10  non_boundary      198569 non-null  int64 
 11  is_wicket         198569 non-null  int64 
 12  dismissal_kind    9764 non-null    object
 13  player_dismissed  9764 non-null    object
 14  fielder           6968 non-null    object
 15  extras_type       10500 non-null   object
 16  batting_team      198569 non-null  objec

In [15]:
batsmen_score = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs"})
batsmen_scores_30 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=30).astype(int)).\
    rename(columns={"batsman_runs": "total_runs_30"})
batsmen_scores_50 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=50).astype(int)).\
    rename(columns={"batsman_runs": "total_runs_50"})
batsmen_scores_100 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=100).astype(int)).\
    rename(columns={"batsman_runs": "total_runs_100"})
batsmen_scores_0 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())==0).astype(int)).\
    rename(columns={"batsman_runs": "total_runs_0"})
batsmen_ball_faced = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['over'].count()).\
    rename(columns={"over": "total_balls_faced"})
batsmen_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'batsman'])['ball'].count()).\
    rename(columns={"ball": "total_legal_balls_faced"})
batsmen_scores6 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 6].groupby(['id', 'batsman'])['batsman_runs'].count()).\
    rename(columns={"batsman_runs": "run_6"})
batsmen_scores4 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 4].groupby(['id', 'batsman'])['batsman_runs'].count()).\
    rename(columns={"batsman_runs": "run_4"})
batsmen_position = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['is_wicket'].min())


df_list = [batsmen_score, batsmen_ball_faced, batsmen_ball_faced_legal, batsmen_scores6, batsmen_scores4, batsmen_position,batsmen_scores_30,batsmen_scores_50,batsmen_scores_100,batsmen_scores_0]
batsmen_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
batsmen_summary = pd.merge(batsmen_summary, matchdata[['id', 'batsman', 'inning', 'batting_team', 'bowling_team']].
                            drop_duplicates(), on=['id', 'batsman'], how='left')
batsmen_summary.rename(columns={'inning': 'batsmen_innings', 'batsman': 'playername', 'batting_team': 'Playing Team',
                                'bowling_team': 'Opponent Team'}, inplace=True)
       

In [16]:
# 0, 0, 1
# Match. Batsman      is_wicket
# 123.    AA Noffke       0
# 123.    AA Noffke       0
# 123.    AA Noffke       1

pd.DataFrame(matchdata.groupby(['id', 'batsman'])['is_wicket'].max())

is_wicket
id      batsman                
335982  AA Noffke             1
        B Akhil               1
        BB McCullum           0
        CL White              1
        DJ Hussey             1
...                         ...
1254078 N Rana                1
        RA Tripathi           1
        Ravi Bishnoi          1
        SP Narine             1
        Shubman Gill          1

[12491 rows x 1 columns]

In [17]:
bowler_wickets = pd.DataFrame(matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count()).\
    rename(columns={"is_wicket": "total_wickets"})
bowler_overs_bow = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['over'].count()).\
    rename(columns={"over": "total_balls_bowled"})
bowler_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'bowler'])['ball'].count()).\
    rename(columns={"ball": "total_legal_balls_bowled"})
# calculating the number of maiden overs bowled in a match
bowler_runs_given = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['batsman_runs'].sum()).\
    rename(columns={"batsman_runs": "total_runs_given"})
#matchdata_runs_per_over = pd.DataFrame(self.matchdata.groupby(['matchid', 'bowlername', 'over_num'])['batsmanscorevalue'].sum()).reset_index()
#matchdata_runs_per_over['maiden_overs'] = np.where(matchdata_runs_per_over['batsmanscorevalue'] == 0, 1, 0)
#matchdata_runs_per_over = matchdata_runs_per_over.groupby(['matchid', 'bowlername'])['maiden_overs'].sum()
# combining all the features created so far
df_list = [bowler_wickets, bowler_overs_bow, bowler_ball_faced_legal, bowler_runs_given]
bowler_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
# adding additional columns just in case we need for modeling
bowler_summary = pd.merge(bowler_summary, matchdata[['id', 'bowler', 'inning', 'batting_team', 'bowling_team']].drop_duplicates(), on=['id', 'bowler'], how='left')
bowler_summary.rename(columns={'inning': 'bowlers_innings', 'bowler': 'playername', 'batting_team': 'Playing team',
                                'bowling_team': 'Opponent Team'}, inplace=True)
bowler_summary['economy_rate'] = (bowler_summary['total_runs_given'] * 6) / bowler_summary['total_legal_balls_bowled']

In [33]:
ipl_merged_scorecard = pd.merge(batsmen_summary, bowler_summary, on=['id', 'playername'], how='outer')

In [19]:
MINAVGBALLSFACED = 8
MINAVGBOWLSBOWLED = 6
player_avg = ipl_merged_scorecard[['playername', 'total_balls_faced', 'total_balls_bowled']].fillna(0)
player_avg = pd.DataFrame(player_avg.groupby('playername')[['total_balls_faced', 'total_balls_bowled']].mean())
conditions = [((player_avg['total_balls_faced'] >= MINAVGBALLSFACED) & (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)),
              (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)]
choices = ['AllRounder', 'Bowler']
player_avg['playing_role'] = np.select(conditions, choices, default='Batsmen')
player_avg = player_avg.reset_index()

In [20]:
fielder_data = pd.DataFrame(matchdata[((matchdata['is_wicket'] == 1) & (matchdata['dismissal_kind'].isin(['caught','caught and bowled','run out','stumped'])))].
                                      groupby(['fielder','dismissal_kind'])['is_wicket'].count()).\
            rename(columns={"is_wicket": "Fielding_stats"})

In [21]:
# Checking for null values
ipl_merged_scorecard.isnull().sum()

id                              0
playername                      0
total_runs                   5106
total_balls_faced            5106
total_legal_balls_faced      5157
run_6                       12939
run_4                        9949
is_wicket                    5106
total_runs_30                5106
total_runs_50                5106
total_runs_100               5106
total_runs_0                 5106
batsmen_innings              5106
Playing Team                 5106
Opponent Team_x              5119
total_wickets               11995
total_balls_bowled           7678
total_legal_balls_bowled     7678
total_runs_given             7678
bowlers_innings              7678
Playing team                 7678
Opponent Team_y              7690
economy_rate                 7678
dtype: int64

In [22]:
#id                              0
#playername                      0

# Null -> 0
#----------------------------------
#total_runs                   5106
#total_balls_faced            5106
#total_legal_balls_faced      5157
#run_6                       12939
#run_4                        9949
#total_runs_30                5106
#total_runs_50                5106
#total_runs_100               5106
#total_runs_0                 5106
#total_wickets               11995
#total_balls_bowled           7678
#total_legal_balls_bowled     7678
#total_runs_given             7678
#economy_rate                 7678

#----------------------------------

# Batsman_summary
# Not in Bowler_summary - join -> Null - 0?
#is_wicket                    5106
#batsmen_innings              5106

#----------------------------------
# Bowler_summary
# Not in Batsman_summary - join -> Null - 0?
#bowlers_innings              7678

#----------------------------------
# Swap for batmen who didnt bowl OR bowlers who didnt bat?
#Playing Team                 5106
#Opponent Team_x              5119
#Playing team                 7678
#Opponent Team_y              7690


In [23]:
# List of cols where null -> 0
list_of_cols = ipl_merged_scorecard.columns[2:]
list_of_cols = list(list_of_cols)
#list_of_cols.remove("")
list_of_cols

['total_runs',
 'total_balls_faced',
 'total_legal_balls_faced',
 'run_6',
 'run_4',
 'is_wicket',
 'total_runs_30',
 'total_runs_50',
 'total_runs_100',
 'total_runs_0',
 'batsmen_innings',
 'Playing Team',
 'Opponent Team_x',
 'total_wickets',
 'total_balls_bowled',
 'total_legal_balls_bowled',
 'total_runs_given',
 'bowlers_innings',
 'Playing team',
 'Opponent Team_y',
 'economy_rate']

In [24]:
# in case all cols Nulls -> 0
for col in list_of_cols:
  #ipl_merged_scorecard.loc[ ipl_merged_scorecard[col].isnull() , col ] = 0
  ipl_merged_scorecard[col].fillna(0, inplace=True)
ipl_merged_scorecard.isnull().sum()

id                          0
playername                  0
total_runs                  0
total_balls_faced           0
total_legal_balls_faced     0
run_6                       0
run_4                       0
is_wicket                   0
total_runs_30               0
total_runs_50               0
total_runs_100              0
total_runs_0                0
batsmen_innings             0
Playing Team                0
Opponent Team_x             0
total_wickets               0
total_balls_bowled          0
total_legal_balls_bowled    0
total_runs_given            0
bowlers_innings             0
Playing team                0
Opponent Team_y             0
economy_rate                0
dtype: int64

In [25]:
# Swap for batmen who didnt bowl OR bowlers who didnt bat?
#Playing Team                 5106
#Opponent Team_x              5119
#Playing team                 7678
#Opponent Team_y              7690
team_cols = ["Playing Team", "Opponent Team_x", "Playing team", "Opponent Team_y" ]
ipl_merged_scorecard[["Playing Team", "Opponent Team_x"]].isnull().sum()


Playing Team       0
Opponent Team_x    0
dtype: int64

In [26]:
ipl_merged_scorecard[["Playing team", "Opponent Team_y"]].isnull().sum()

Playing team       0
Opponent Team_y    0
dtype: int64

In [27]:
ipl_merged_scorecard [ (ipl_merged_scorecard["Playing Team" ].isna() == False) & (ipl_merged_scorecard["Opponent Team_x"].isna()) ]

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,...,Playing Team,Opponent Team_x,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,bowlers_innings,Playing team,Opponent Team_y,economy_rate


In [28]:
ipl_merged_scorecard[ ipl_merged_scorecard["id"] == 501265]

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,...,Playing Team,Opponent Team_x,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,bowlers_innings,Playing team,Opponent Team_y,economy_rate
3635,501265,CA Ingram,13.0,12.0,12.0,0.0,1.0,0.0,0.0,0.0,...,Delhi Daredevils,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
3636,501265,DA Warner,19.0,10.0,10.0,2.0,1.0,0.0,0.0,0.0,...,Delhi Daredevils,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
3637,501265,MS Wade,11.0,20.0,19.0,0.0,0.0,0.0,0.0,0.0,...,Delhi Daredevils,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
3638,501265,NV Ojha,8.0,20.0,20.0,0.0,1.0,0.0,0.0,0.0,...,Delhi Daredevils,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
3639,501265,Y Venugopal Rao,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Delhi Daredevils,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
13867,501265,AC Thomas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,18.0,18.0,15.0,1.0,Delhi Daredevils,0,5.000000
13868,501265,JP Faulkner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1.0,12.0,12.0,9.0,1.0,Delhi Daredevils,0,4.500000
13869,501265,MR Marsh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1.0,13.0,13.0,11.0,1.0,Delhi Daredevils,0,5.076923
13870,501265,R Sharma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,6.0,6.0,4.0,1.0,Delhi Daredevils,0,4.000000
13871,501265,SS Mundhe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1.0,7.0,6.0,5.0,1.0,Delhi Daredevils,0,5.000000


In [29]:
ipl_merged_scorecard[ ipl_merged_scorecard["id"] == 829763]

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,...,Playing Team,Opponent Team_x,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,bowlers_innings,Playing team,Opponent Team_y,economy_rate
7296,829763,AB de Villiers,57.0,47.0,45.0,1.0,9.0,0.0,1.0,1.0,...,Royal Challengers Bangalore,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
7297,829763,CH Gayle,10.0,4.0,4.0,1.0,1.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
7298,829763,D Wiese,11.0,9.0,9.0,0.0,2.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
7299,829763,HV Patel,6.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
7300,829763,KD Karthik,27.0,19.0,17.0,1.0,3.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
7301,829763,Mandeep Singh,27.0,21.0,20.0,0.0,4.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
7302,829763,SN Khan,45.0,22.0,19.0,1.0,6.0,0.0,1.0,0.0,...,Royal Challengers Bangalore,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
7303,829763,V Kohli,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000
15358,829763,DS Kulkarni,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1.0,26.0,24.0,34.0,1.0,Royal Challengers Bangalore,0,8.500000
15359,829763,JP Faulkner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,13.0,12.0,25.0,1.0,Royal Challengers Bangalore,0,12.500000


In [30]:
matches2020=pd.read_csv('C:/Users/Admin/Desktop/BYOP Project/IPL Matches 2008-2020.csv')
matches2021=pd.read_csv('C:/Users/Admin/Desktop/BYOP Project/IPL Matches Dataset 2021.csv')

In [31]:
# Merging the matches dataset from 2008 to 2021

matchdetails = pd.concat(
    map(pd.read_csv, ['C:/Users/Admin/Desktop/BYOP Project/IPL Matches 2008-2020.csv', 'C:/Users/Admin/Desktop/BYOP Project/IPL Matches Dataset 2021.csv']))

In [32]:
matchdetails.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 837 entries, 0 to 20
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               837 non-null    int64  
 1   city             824 non-null    object 
 2   date             837 non-null    object 
 3   player_of_match  833 non-null    object 
 4   venue            837 non-null    object 
 5   neutral_venue    837 non-null    int64  
 6   team1            837 non-null    object 
 7   team2            837 non-null    object 
 8   toss_winner      837 non-null    object 
 9   toss_decision    837 non-null    object 
 10  winner           833 non-null    object 
 11  result           833 non-null    object 
 12  result_margin    819 non-null    float64
 13  eliminator       833 non-null    object 
 14  method           19 non-null     object 
 15  umpire1          837 non-null    object 
 16  umpire2          837 non-null    object 
dtypes: float64(1), in

In [35]:
ipl_merged_scorecard.head()

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,...,Playing Team,Opponent Team_x,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,bowlers_innings,Playing team,Opponent Team_y,economy_rate
0,335982,AA Noffke,9.0,12.0,11.0,NaN,1.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,Kolkata Knight Riders,1.0,25.0,24.0,35.0,1.0,Kolkata Knight Riders,Royal Challengers Bangalore,8.75
1,335982,B Akhil,0.0,2.0,2.0,NaN,NaN,0.0,0.0,0.0,...,Royal Challengers Bangalore,Kolkata Knight Riders,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,BB McCullum,158.0,77.0,73.0,13.0,10.0,0.0,1.0,1.0,...,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,CL White,6.0,10.0,8.0,NaN,NaN,0.0,0.0,0.0,...,Royal Challengers Bangalore,Kolkata Knight Riders,NaN,7.0,6.0,22.0,1.0,Kolkata Knight Riders,Royal Challengers Bangalore,22.00
4,335982,DJ Hussey,12.0,12.0,12.0,NaN,1.0,0.0,0.0,0.0,...,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
#separating the year column

matchdetails['year'] = matchdetails['date'].str[:4] 

In [37]:
matchdetails

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,year
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,2008
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri,2008
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar,2008
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper,2008
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,1254074,Chennai,2021-04-23,KL Rahul,"MA Chidambaram Stadium, Chepauk, Chennai",0,Mumbai Indians,Punjab Kings,Punjab Kings,field,Punjab Kings,wickets,9.0,N,NaN,C Shamshuddin,Nitin Menon,2021
17,1254075,Mumbai,2021-04-24,CH Morris,"Wankhede Stadium, Mumbai",0,Kolkata Knight Riders,Rajasthan Royals,Rajasthan Royals,field,Rajasthan Royals,wickets,6.0,N,NaN,Navdeep Singh,S Ravi,2021
18,1254076,Mumbai,2021-04-25,RA Jadeja,"Wankhede Stadium, Mumbai",0,Chennai Super Kings,Royal Challengers Bangalore,Chennai Super Kings,bat,Chennai Super Kings,runs,69.0,N,NaN,AK Chaudhary,VK Sharma,2021
19,1254077,Chennai,2021-04-25,PP Shaw,"MA Chidambaram Stadium, Chepauk, Chennai",0,Delhi Capitals,Sunrisers Hyderabad,Delhi Capitals,bat,Delhi Capitals,tie,NaN,Y,NaN,CB Gaffaney,KN Ananthapadmanabhan,2021


In [77]:
# Merging the ball by ball revised data and matches dataset

finaldata = pd.merge(ipl_merged_scorecard, matchdetails, on='id')

In [78]:
finaldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17597 entries, 0 to 17596
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        17597 non-null  int64  
 1   playername                17597 non-null  object 
 2   total_runs                12491 non-null  float64
 3   total_balls_faced         12491 non-null  float64
 4   total_legal_balls_faced   12440 non-null  float64
 5   run_6                     4658 non-null   float64
 6   run_4                     7648 non-null   float64
 7   is_wicket                 12491 non-null  float64
 8   total_runs_30             12491 non-null  float64
 9   total_runs_50             12491 non-null  float64
 10  total_runs_100            12491 non-null  float64
 11  total_runs_0              12491 non-null  float64
 12  batsmen_innings           12491 non-null  float64
 13  Playing Team              12491 non-null  object 
 14  Oppone

In [79]:
finaldata.isnull().sum()

id                              0
playername                      0
total_runs                   5106
total_balls_faced            5106
total_legal_balls_faced      5157
run_6                       12939
run_4                        9949
is_wicket                    5106
total_runs_30                5106
total_runs_50                5106
total_runs_100               5106
total_runs_0                 5106
batsmen_innings              5106
Playing Team                 5106
Opponent Team_x              5119
total_wickets               11995
total_balls_bowled           7678
total_legal_balls_bowled     7678
total_runs_given             7678
bowlers_innings              7678
Playing team                 7678
Opponent Team_y              7690
economy_rate                 7678
city                          278
date                            0
player_of_match                59
venue                           0
neutral_venue                   0
team1                           0
team2         

In [50]:
finaldata.to_csv(r'C:\Users\Admin\Desktop\BYOP Project\finaldata1.csv', index = False)

In [80]:
# Replacing the null values with 0
finaldata.fillna(value = 0,
          inplace = True)

In [85]:
cols = ["Playing Team", "Opponent Team_x", "Playing team", "Opponent Team_y", "economy_rate", "city", "player_of_match", "winner", "result", "result_margin", "eliminator", "method"]

finaldata[cols] = finaldata[cols].replace(['0', 0], np.nan)


In [86]:
finaldata.isnull().sum()

id                              0
playername                      0
total_runs                      0
total_balls_faced               0
total_legal_balls_faced         0
run_6                           0
run_4                           0
is_wicket                       0
total_runs_30                   0
total_runs_50                   0
total_runs_100                  0
total_runs_0                    0
batsmen_innings                 0
Playing Team                 5106
Opponent Team_x              5119
total_wickets                   0
total_balls_bowled              0
total_legal_balls_bowled        0
total_runs_given                0
bowlers_innings                 0
Playing team                 7678
Opponent Team_y              7690
economy_rate                 7689
city                          278
date                            0
player_of_match                59
venue                           0
neutral_venue                   0
team1                           0
team2         

In [ ]:
Year
Player of the match (max)
Total Runs


match id
player of the match
runs
year



